# ყველა ფაილის მისამართის ამოღება

In [2]:
import glob
import os

In [9]:
path = '*.m4a'
files = glob.glob(path)
# files

# audio ფაილების გადაყვანა wav-ში

In [8]:
from pydub import AudioSegment
src = files[0]
# dst = "test.wav"
dst = src[:-3] + "wav"
dst


'/home/mariam/Desktop/levan-gela-1g.wav'

In [27]:
s = AudioSegment.from_file(src)
s.export(dst, format="wav")
# s

<_io.BufferedRandom name='test.wav'>

In [16]:
import pandas as pd
import os
import librosa
import librosa.display

In [57]:
import pandas as pd
import os
import librosa
# უბრალოდ შევამოწმე რას აკეთებდა

features = []
data = extract_features("test.wav")
data2 = extract_features("devi-khos-1a.wav")
label = "1"
features.append([data, label])
features.append([data2, label])
datafr = pd.DataFrame(features, columns=['feature','class_label' ])
datafr

,feature,class_label
0,"[-451.63174, 57.256325, 8.637601, 20.835255, 0...",1
1,"[-475.20328, 96.31608, 13.486313, 19.195719, 6...",1


##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [42]:
import numpy as np

def get_features(file_name):
   
    try:
        librosa.load
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [54]:
def get_all_features():
    folders = os.listdir('ML-FinalProjectVoices')
    folders.sort()
    features = []
    for label, sub_dir in enumerate(folders):  
        for file_name in glob.glob(os.path.join('ML-FinalProjectVoices',sub_dir,"*.wav")):
            print("Extracting file ", file_name)
            try:
                mfccs = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            features.append([mfccs,label])
    data = pd.DataFrame(features, columns=['feature', 'class_label'])
    return data

In [58]:
# get_all_features()